In [2]:
!pip install flask flask-cors pyngrok newspaper3k lime tensorflow nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 79.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 10.2 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=ef45908517408194f2857d1ef492cc7085ceb10e3234cb8390d030404edb507d
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=391e7b0845cd16804b36ba975acc8105a6803531e7

In [6]:
!pip uninstall -y lxml
!pip install lxml==4.9.3
!pip install newspaper3k
!pip install lxml_html_clean


Found existing installation: lxml 5.3.2
Uninstalling lxml-5.3.2:
  Successfully uninstalled lxml-5.3.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 46.4 MB/s eta 0:00:00


In [2]:
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 83.6 MB/s eta 0:00:00


In [4]:
!ngrok config add-authtoken "2vMiRmqM75KyG7V5xKNGOelIZ78_EkWAHZkZRzLB4mqRroVt"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from newspaper import Article
from lime.lime_text import LimeTextExplainer
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import re
import nltk
import numpy as np
import requests
import fitz  # PyMuPDF for PDF reading
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from pyngrok import ngrok

# NLTK setup
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Flask setup
app = Flask(__name__)
CORS(app)

# Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if w not in stop_words]
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(w) for w in tokens]
    return " ".join(stemmed_tokens)

# Load model and tokenizer
model = load_model("lstm_model.h5")
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

MAXLEN = 200

# LIME explanation function
def explain_with_lime(model, tokenizer, text, maxlen=MAXLEN):
    explainer = LimeTextExplainer(class_names=["fake", "real"])

    def predict_proba(texts):
        sequences = tokenizer.texts_to_sequences(texts)
        padded = pad_sequences(sequences, maxlen=maxlen)
        probs = model.predict(padded, verbose=0)
        return np.hstack([1 - probs, probs])

    exp = explainer.explain_instance(text, predict_proba, num_features=10)
    return [word for word, _ in exp.as_list()]

# ✅ NEW: Extract text from a PDF URL
def extract_text_from_pdf(url):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/122.0.0.0 Safari/537.36"
        )
    }
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise ValueError("Failed to download PDF.")

    with open("temp.pdf", "wb") as f:
        f.write(response.content)

    text = ""
    doc = fitz.open("temp.pdf")
    for page in doc:
        text += page.get_text()
    return text

# Main API endpoint
@app.route("/analyze", methods=["POST"])
def analyze():
    try:
        url = request.json.get("url")
        if not url or not url.startswith("http"):
            return jsonify({"error": "Invalid or missing URL"}), 400

        headers = {
            "User-Agent": (
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/122.0.0.0 Safari/537.36"
            )
        }

        # ✅ Use different parsing methods based on URL type
        if url.lower().endswith(".pdf"):
            try:
                raw_text = extract_text_from_pdf(url)
            except Exception as e:
                return jsonify({"error": f"PDF extraction failed: {str(e)}"}), 400
        else:
            article = Article(url, request_headers=headers)
            try:
                article.download()
                article.parse()
                raw_text = article.text
            except Exception as e:
                return jsonify({"error": f"Failed to fetch article: {str(e)}"}), 400

        if not raw_text.strip():
            return jsonify({"error": "Could not extract article text."}), 400

        # Preprocess + predict
        processed = preprocess_text(raw_text)
        sequence = tokenizer.texts_to_sequences([processed])
        padded = pad_sequences(sequence, maxlen=MAXLEN)
        prob = model.predict(padded, verbose=0)[0][0]
        credibility = "High" if prob < 0.5 else "Low"

        keywords = explain_with_lime(model, tokenizer, raw_text)

        return jsonify({
            "probability": float(prob),
            "credibility": credibility,
            "keywords": keywords
        })

    except Exception as e:
        print(f"❌ Error occurred: {e}")
        return jsonify({"error": str(e)}), 500

# Start server using ngrok
public_url = ngrok.connect(5000)
print(f"🔗 Your public API URL: {public_url}/analyze")
app.run(port=5000)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


🔗 Your public API URL: NgrokTunnel: "https://8d54-35-221-182-198.ngrok-free.app" -> "http://localhost:5000"/analyze
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 04:09:29] "OPTIONS / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 04:09:54] "OPTIONS / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 04:10:04] "OPTIONS / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 04:10:22] "OPTIONS / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 04:12:57] "OPTIONS /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Apr/2025 04:13:23] "POST /analyze HTTP/1.1" 200 -
